In [1]:
import pandas as pd

In [2]:
data= pd.read_csv('onehr.data')

In [3]:
data

,Date,WSR0,WSR1,WSR2,WSR3,WSR4,WSR5,WSR6,WSR7,WSR8,...,RH50,U50,V50,HT50,KI,TT,SLP,SLP_,Precp,class
0,1/1/1998,0.8,1.8,2.4,2.1,2,2.1,1.5,1.7,1.9,...,0.15,10.67,-1.56,5795,-12.1,17.9,10330,-55,0,0.0
1,1/2/1998,2.8,3.2,3.3,2.7,3.3,3.2,2.9,2.8,3.1,...,0.48,8.39,3.84,5805,14.05,29,10275,-55,0,0.0
2,1/3/1998,2.9,2.8,2.6,2.1,2.2,2.5,2.5,2.7,2.2,...,0.6,6.94,9.8,5790,17.9,41.3,10235,-40,0,0.0
3,1/4/1998,4.7,3.8,3.7,3.8,2.9,3.1,2.8,2.5,2.4,...,0.49,8.73,10.54,5775,31.15,51.7,10195,-40,2.08,0.0
4,1/5/1998,2.6,2.1,1.6,1.4,0.9,1.5,1.2,1.4,1.3,...,?,?,?,?,?,?,?,?,0.58,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2531,12/27/2004,0.3,0.4,0.5,0.5,0.2,0.3,0.4,0.4,1.3,...,0.07,7.93,-4.41,5800,-25.6,21.8,10295,65,0,0.0
2532,12/28/2004,1,1.4,1.1,1.7,1.5,1.7,1.8,1.5,2.1,...,0.04,5.95,-1.14,5845,-19.4,19.1,10310,15,0,0.0
2533,12/29/2004,0.8,0.8,1.2,0.9,0.4,0.6,0.8,1.1,1.5,...,0.06,7.8,-0.64,5845,-9.6,35.2,10275,-35,0,0.0
2534,12/30/2004,1.3,0.9,1.5,1.2,1.6,1.8,1.1,1,1.9,...,0.25,7.72,-0.89,5845,-19.6,34.2,10245,-30,0.05,0.0


In [4]:
from kedro.pipeline import node

def load_data(data):
    return data
nodes = []

nodes.append(node(func=load_data,inputs='ozone',outputs='raw_data'))

In [5]:
import numpy as np

def preprocessing(data):
    data = data.replace('?', np.nan)

    data['Date'] = pd.to_datetime(data['Date'])
    
    for column in data.columns[1:-2]:
        data[column] = data[column].astype(float)
          
    data = data.drop_duplicates()
    data.reset_index(drop=True, inplace=True)

    data.fillna(data.mean(), inplace=True)
    
    return data

nodes.append(
    node(func=preprocessing,
         inputs="raw_data",
         outputs="ozone_processed")
)

In [6]:
from kedro.io import DataCatalog
from kedro.extras.datasets.pandas import CSVDataSet

data_catalog = DataCatalog(
    {
        "ozone": CSVDataSet(filepath="onehr.data",
                            load_args=dict(sep=","))
    },
    {
        "ozone_processed": CSVDataSet(filepath="onehr_processed.data",
                                      save_args=dict(sep=',', index=False))
    }
)

In [7]:
from kedro.pipeline import Pipeline

pipeline = Pipeline(nodes)

In [8]:
from kedro.runner import SequentialRunner

runner = SequentialRunner()
runner.run(pipeline,data_catalog)

C:\Users\User\anaconda3\lib\site-packages\flask\json\__init__.py:31: DeprecationWarning: Importing 'itsdangerous.json' is deprecated and will be removed in ItsDangerous 2.1. Use Python's 'json' module instead.
  _slash_escape = "\\/" not in _json.dumps("/")
C:\Users\User\anaconda3\lib\site-packages\flask\json\__init__.py:61: DeprecationWarning: Importing 'itsdangerous.json' is deprecated and will be removed in ItsDangerous 2.1. Use Python's 'json' module instead.
  class JSONEncoder(_json.JSONEncoder):
C:\Users\User\anaconda3\lib\site-packages\flask\json\__init__.py:103: DeprecationWarning: Importing 'itsdangerous.json' is deprecated and will be removed in ItsDangerous 2.1. Use Python's 'json' module instead.
  class JSONDecoder(_json.JSONDecoder):


[08/02/22 19:28:08] INFO     Loading data from 'ozone' (CSVDataSet)...                          ]8;id=957366;file://C:\Users\User\anaconda3\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=348283;file://C:\Users\User\anaconda3\lib\site-packages\kedro\io\data_catalog.py#343\343]8;;\

                    INFO     Running node: load_data([ozone]) -> [raw_data]                             ]8;id=55101;file://C:\Users\User\anaconda3\lib\site-packages\kedro\pipeline\node.py\node.py]8;;\:]8;id=487774;file://C:\Users\User\anaconda3\lib\site-packages\kedro\pipeline\node.py#327\327]8;;\

                    INFO     Saving data to 'raw_data' (MemoryDataSet)...                       ]8;id=942520;file://C:\Users\User\anaconda3\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=12899;file://C:\Users\User\anaconda3\lib\site-packages\kedro\io\data_catalog.py#382\382]8;;\

                    INFO     Completed 1 out of 2 tasks                                     ]8;id=303882;file://C:\Users\User\anaconda3\lib\site-packages\kedro\runner\sequential_runner.py\sequential_runner.py]8;;\:]8;id=261123;file://C:\Users\User\anaconda3\lib\site-packages\kedro\runner\sequential_runner.py#85\85]8;;\

...

                    WARNING  C:\Users\User\AppData\Local\Temp/ipykernel_82832/1957090456.py:14:     ]8;id=657674;file://C:\Users\User\anaconda3\lib\warnings.py\warnings.py]8;;\:]8;id=831943;file://C:\Users\User\anaconda3\lib\warnings.py#109\109]8;;\
                             FutureWarning: Dropping of nuisance columns in DataFrame reductions                   
                             (with 'numeric_only=None') is deprecated; in a future version this                    
                             will raise TypeError.  Select only valid columns before calling the                   
                             reduction.                                                                            
                               data.fillna(data.mean(), inplace=True)                                              
                                                                                                                   

                    INFO     Saving data to 'ozone_processed' (CSVDataSet)...                   ]8;id=982273;file://C:\Users\User\anaconda3\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=497260;file://C:\Users\User\anaconda3\lib\site-packages\kedro\io\data_catalog.py#382\382]8;;\

                    INFO     Completed 2 out of 2 tasks                                     ]8;id=865335;file://C:\Users\User\anaconda3\lib\site-packages\kedro\runner\sequential_runner.py\sequential_runner.py]8;;\:]8;id=582542;file://C:\Users\User\anaconda3\lib\site-packages\kedro\runner\sequential_runner.py#85\85]8;;\

                    INFO     Pipeline execution completed successfully.                                ]8;id=965224;file://C:\Users\User\anaconda3\lib\site-packages\kedro\runner\runner.py\runner.py]8;;\:]8;id=732123;file://C:\Users\User\anaconda3\lib\site-packages\kedro\runner\runner.py#89\89]8;;\

{}

In [13]:
data = pd.read_csv('onehr_processed.data',parse_dates=['Date'])

In [15]:
import hvplot.pandas
import panel as pn
pn.extension()

cols = list(data.columns)
cols.remove('Date')
cols.remove('class')

y=pn.widgets.Select(name='y',options=cols,size=10)

plot = data.hvplot(x='Date',y=y)

pn.Row(y,plot)

[08/02/22 19:30:17] WARNING  C:\Users\User\anaconda3\lib\site-packages\panel\pane\base.py:38:       ]8;id=682895;file://C:\Users\User\anaconda3\lib\warnings.py\warnings.py]8;;\:]8;id=886143;file://C:\Users\User\anaconda3\lib\warnings.py#109\109]8;;\
                             DeprecationWarning: panel.Pane(...) is deprecated, use                                
                             panel.panel(...) instead.                                                             
                               warnings.warn(                                                                      
                                                                                                                   

                    WARNING  C:\Users\User\anaconda3\lib\site-packages\bokeh\core\property\bases.py ]8;id=259348;file://C:\Users\User\anaconda3\lib\warnings.py\warnings.py]8;;\:]8;id=585623;file://C:\Users\User\anaconda3\lib\warnings.py#109\109]8;;\
                             :259: DeprecationWarning: elementwise comparison failed; this will                    
                             raise an error in the future.                                                         
                               return new == old                                                                   
                                                                                                                   

                    WARNING  C:\Users\User\anaconda3\lib\site-packages\bokeh\core\property\bases.py ]8;id=927699;file://C:\Users\User\anaconda3\lib\warnings.py\warnings.py]8;;\:]8;id=830000;file://C:\Users\User\anaconda3\lib\warnings.py#109\109]8;;\
                             :259: DeprecationWarning: elementwise comparison failed; this will                    
                             raise an error in the future.                                                         
                               return new == old                                                                   
                                                                                                                   

In [16]:
plot = data.hvplot(x='Date',y=y,kind='kde')

pn.Row(y,plot)

[08/02/22 19:30:22] WARNING  C:\Users\User\anaconda3\lib\site-packages\panel\pane\base.py:38:       ]8;id=100662;file://C:\Users\User\anaconda3\lib\warnings.py\warnings.py]8;;\:]8;id=555033;file://C:\Users\User\anaconda3\lib\warnings.py#109\109]8;;\
                             DeprecationWarning: panel.Pane(...) is deprecated, use                                
                             panel.panel(...) instead.                                                             
                               warnings.warn(                                                                      
                                                                                                                   

In [17]:
by = pn.widgets.Select(name='by', options=[None, 'class'])

plot = data.hvplot.kde(y=y, by=by)

pn.Row(pn.Column(y, by), plot)

[08/02/22 19:30:23] WARNING  C:\Users\User\anaconda3\lib\site-packages\panel\pane\base.py:38:       ]8;id=780419;file://C:\Users\User\anaconda3\lib\warnings.py\warnings.py]8;;\:]8;id=264207;file://C:\Users\User\anaconda3\lib\warnings.py#109\109]8;;\
                             DeprecationWarning: panel.Pane(...) is deprecated, use                                
                             panel.panel(...) instead.                                                             
                               warnings.warn(                                                                      
                                                                                                                   